In [1]:
from openai import OpenAI
from config.open_ai.dps_qc_report import get_qc_prompt

prompt = get_qc_prompt()

client = OpenAI()

assistant = client.beta.assistants.create(
  name="Financial Analyst Assistant",
  instructions=prompt,
  model="gpt-4o",
  tools=[{"type": "file_search"}],
)

In [2]:
# Create a vector store caled "DPS items"
vector_store = client.beta.vector_stores.create(name="DPS items")

from config.settings import DATA_DIR
import os

file_dir = DATA_DIR / 'qc_data' / 'qc_templates'
files = os.listdir(file_dir)
file_path = file_dir / files[-2]

# Ready the files for upload to OpenAI
file_paths = [file_path]
file_streams = [open(path, "rb") for path in file_paths]

# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)

# You can print the status and the file counts of the batch to see the result of this operation.
print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)


In [3]:
assistant = client.beta.assistants.update(
  assistant_id=assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

In [4]:
# Upload the user provided file to OpenAI
message_file = client.files.create(
  file=open(file_path, "rb"), purpose="assistants"
)

# Create a thread and attach the file to the message
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": prompt,
      # Attach the new file to the message.
      "attachments": [
        { "file_id": message_file.id, "tools": [{"type": "file_search"}] }
      ],
    }
  ]
)

# The thread now has a vector store with that file in its tool resources.
print(thread.tool_resources.file_search)

ToolResourcesFileSearch(vector_store_ids=['vs_67a9d4d4675081919fcc40ffe21d72eb'])


In [6]:
import json

# Fetch messages from the thread
messages = client.beta.threads.messages.list(thread_id=thread.id)

# Extract content from messages
for msg in messages:
    role = msg.role
    content = msg.content

In [ ]:
print(content[0].text.value)

You are a quality control report extractor. Your task is to extract structured quality control details from the provided pdf and output a list of valid JSON objects, each conforming to the given schema.

Leave the first few pages which contains summary report. Go to the next pages each page has 1 item and leave the pages which contains images.### **Required Fields and Their Meanings:**
- **product_name** (string): The name of the product.
- **RAG** (string): The RAG value (e.g., "BLUE", "AMBER", "GREEN").
- **expected_qty** (string): The expected quantity.
- **received_qty** (string): The received quantity.
- **supplier_code** (string): The supplier code.
- **supplier** (string): The supplier name.
- **coo** (string): Country of Origin.
- **received_date** (string): Date/time when received, in "YYYY-MM-DD HH:MM:SS" format.
- **inspection_date** (string): Date/time of inspection, in "YYYY-MM-DD HH:MM:SS" format.
- **print_date** (string): The print date, in "YYYY-MM-DD" format.
- **iss_